In [1]:
versioninfo()


Julia Version 0.4.1


In [1]:
using SymPy
using PyCall


In [123]:
module QNET
using SymPy
using PyCall

@pyimport qnet.algebra.circuit_algebra as ca
import Base:show, writemime, (*), (+), (-), (/), (<<), (|), (==), promote_rule, convert, getindex
import SymPy:latex


export QSym, Create, Destroy, LocalSigma, dag, simplify, expand, substitute, operands

type QSym{T} <: Number
    x::PyObject
end
QSym(T::Symbol, x::PyObject) = QSym{T}(x)
QSym(x::PyObject) = QSym(Symbol(x[:__class__][:__name__]), x)
QSym(x::Symbol; kwargs...) = QSym(SymPy.symbols(string(x); kwargs...).x)

## text/latex -- for IJulia
function latex(s::QSym)
    ca.tex(s.x)
end
writemime(io::IO, ::MIME"text/latex", x::QSym) = print(io, "\$" * latex(x) * "\$")
# writemime{N}(io::IO, ::MIME"text/latex", x::NTuple{N,QSym}) = print(io, "\$\\left(" * join(map(latex,x), ",\\;") * "\\right)\$")
writemime{S<:QSym}(io::IO, ::MIME"text/latex", x::Vector{S}) = print(io, "\$\\left[" * join(map(latex,x), ",\\;") * "\\right]\$")


NotImplemented = pyeval("NotImplemented")

QSymTypes = Union{QSym,Symbol,Number,PyObject}

for op in [:(*), :(<<), :(-), :(/), :(+), :(|), :(==)]
    opstr = "a $op b"
    @eval function $op(a::QSym, b::QSym)
        res = pyeval($opstr; a=a.x, b=b.x)
        if res == NotImplemented
            error("Cannot compute  $a $op $b")
        end
        if isa(res, PyObject)
            return QSym(res)
        else
            return res
        end
            
    end
    
    @eval $op{T<:QSymTypes, S<:QSym}(a::T, b::S) = $op(promote(a, b)...)
    @eval $op{T<:QSym, S<:QSymTypes}(a::T, b::S) = $op(promote(a, b)...)
end



promote_rule{T<:QSym, S<:Number}(::Type{T}, ::Type{S}) = QSym
promote_rule{T<:QSym}(::Type{T}, ::Type{Symbol}) = QSym
promote_rule{T<:QSym}(::Type{T}, ::Type{PyObject}) = QSym

## Conversion
convert(::Type{QSym}, o::PyCall.PyObject) = QSym(o)
convert(::Type{PyObject}, s::QSym) = s.x
convert{T<:Number}(::Type{QSym}, x::T) = QSym(convert(PyCall.PyObject, x))
convert(::Type{QSym}, x::Symbol) = QSym(x)


getindex(x::QSym, k) = x.x[k]


Create(s) = QSym(ca.Create(s))
Destroy(s) = QSym(ca.Destroy(s))
LocalSigma(s, j, k) = QSym(ca.LocalSigma(s, j, k))



dag(s::QSym) = QSym(s.x[:dag]())
expand(s::QSym)= QSym(s.x[:expand]())
simplify(s::QSym)= QSym(s.x[:simplify_scalar]())
substitute(s::QSym, d::Dict) = QSym(s.x[:substitute](convert(Dict{PyObject, PyObject}, d)))
operands(s::QSym) = [QSym(x) for x=s[:operands]]



function to_csc(o::QSym, full_space=nothing)
    if full_space === nothing
        full_space = ca.space(o.x)
        if full_space == ca.FullSpace || full_space == ca.TrivialSpace
            error("Need to provide a specific space for representation")
        end 
    end
    dim = -1
    try
        dim = full_space[:dimension]
    catch
        error("Need to specify a dimension for all degrees of freedom of space $(full_space[:__str__]())")
    end
    oqt = o[:to_qutip](full_space)
    nnz = oqt[:data][:nnz]
    SparseMatrixCSC(dim, dim, int(oqt[:data][:indptr]+1), int(oqt[:data][:indices]+1), oqt[:data][:data]).'
end

end


QNET

In [124]:
β2 = convert(QNET.QSym, Sym(:beta))
β = QNET.QSym(:beta; real=true)
ad = QNET.Create(1)
expr =  β * ad
expr = (expr + QNET.dag(expr))/2

QNET.QSym{:ScalarTimesOperator}(PyObject ScalarTimesOperator(1/2, OperatorPlus(ScalarTimesOperator(beta, Create(LocalSpace('1', ''))), ScalarTimesOperator(beta, Destroy(LocalSpace('1', ''))))))

0

In [125]:
ad2 = QNET.Create(2)
ad2[:space][:dimension] = 100

100

In [126]:
ad2s = QNET.to_csc(ad2)

100x100 sparse matrix with 99 Complex{Float64} entries:
	[2  ,   1]  =  1.0+0.0im
	[3  ,   2]  =  1.41421+0.0im
	[4  ,   3]  =  1.73205+0.0im
	[5  ,   4]  =  2.0+0.0im
	[6  ,   5]  =  2.23607+0.0im
	[7  ,   6]  =  2.44949+0.0im
	[8  ,   7]  =  2.64575+0.0im
	[9  ,   8]  =  2.82843+0.0im
	[10 ,   9]  =  3.0+0.0im
	[11 ,  10]  =  3.16228+0.0im
	⋮
	[90 ,  89]  =  9.43398+0.0im
	[91 ,  90]  =  9.48683+0.0im
	[92 ,  91]  =  9.53939+0.0im
	[93 ,  92]  =  9.59166+0.0im
	[94 ,  93]  =  9.64365+0.0im
	[95 ,  94]  =  9.69536+0.0im
	[96 ,  95]  =  9.74679+0.0im
	[97 ,  96]  =  9.79796+0.0im
	[98 ,  97]  =  9.84886+0.0im
	[99 ,  98]  =  9.89949+0.0im
	[100,  99]  =  9.94987+0.0im

In [136]:
dim = size(ad2s, 1)
y = zeros(Complex128, dim)
x = randn(dim) + 1im * randn(dim)
@time for k=1:100 At_mul_B!(y, ad2s, x) end
@time for k=1:100 A_mul_B!(y, ad2s, x) end

  0.000158 seconds
  0.000091 seconds


At_mul_B{TA,S,Tx}(A::SparseMatrixCSC{TA,S}, x::Union{DenseArray{Tx,1},SubArray{Tx,1,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at sparse/linalg.jl:76

In [20]:
ad2[:space][:dimension]

LoadError: LoadError: KeyError: dimension not found
while loading In[20], in expression starting on line 1

In [4]:
ad[:space][:dimension] = 10

10

In [39]:
adq = ad[:to_qutip]()

PyObject Quantum object: dims = [[10], [10]], shape = [10, 10], type = oper, isherm = False
Qobj data =
[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 1.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          1.41421356  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          1.73205081  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          2.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          2.23606798  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          2.44948974
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   

In [40]:
adq[:data][:data] |> size, adq[:data][:indices]|> size,  adq[:data][:indptr]|> size

((9,),(9,),(11,))

In [9]:
(expr - (expr |> QNET.dag)) == 0 

true

In [ ]:
1

In [ ]:
QNET.operands(QNET.operands(expr)[2])

In [ ]:
QNET.substitute(expr, Dict(β=>1))

In [ ]:
map(Sym, collect(expr[:all_symbols]()))

In [ ]:
type BLA2{T}
end

In [ ]:
s"n" |> typeof

In [ ]:
s = :(:a)

In [ ]:
q = :(b($s))

In [ ]:
haskey(pn, :tex)

In [ ]:
(QNET.QSym(pn) << QNET.QSym(pn))

In [ ]:
2 * QNET.QSym(QNET.ca.Create(1))

In [ ]:
(QNET.QSym(pn) << QNET.QSym(pn[:series_inverse]()))[:expand]()[:simplify_scalar]()

In [ ]:
(2 *Sym(ca.Create(1))) + Sym(ca.Create(1))

In [ ]:
pn = pnand.PseudoNAND()[:toSLH]()

In [ ]:
ca.Create(1)[:dag]()

In [ ]:
pn[:L][2,1]